In [34]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

In [35]:
import os

def load_from_csv(file_path):
    if not os.path.exists(file_path):
        print("❌ File not found.")
        return

    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"❌ Failed to read CSV: {e}")
        return

    required_columns = {"Date", "Temperature", "Condition"}
    if not required_columns.issubset(set(df.columns)):
      print("❌ Missing required columns.")
      return

    added = 0
    for _, row in df.iterrows():
        date, temp, cond = row["Date"], row["Temperature"], row["Condition"]
        if validate_date(date) and date not in unique_dates:
            try:
                temp = float(temp)
                weather_data.append({"Date": date, "Temperature": temp, "Condition": cond})
                unique_dates.add(date)
                added += 1
            except:
                continue

    with output:
      clear_output()
      print(f"✅ Loaded {added} records from CSV.")

In [36]:
weather_data = []
unique_dates = set()

In [37]:
def validate_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

In [38]:
def add_weather_data(date, temperature, condition):
    if not validate_date(date):
        print("❌ Invalid date format. Use YYYY-MM-DD.")
        return

    if date in unique_dates:
        print("⚠️ Date already exists! Please enter a new date.")
        return

    if not isinstance(temperature, (int, float)):
      print("❌ Temperature must be a number.")
      return

    weather_data.append({"Date": date, "Temperature": temperature, "Condition": condition})
    unique_dates.add(date)

    with output:
      clear_output()
      print("✅ Weather data added successfully.")

In [39]:
def view_weather_data():


  if not weather_data:
    print("No data available.")
    return

  df = pd.DataFrame(weather_data)
  display(df)

def filter_by_conditions_and_date(conditions, start_date, end_date):
    if not weather_data:
        print("No data to filter")
        return pd.DataFrame()

    df = pd.DataFrame(weather_data)
    df["Date"] = pd.to_datetime(df["Date"])

    if conditions:
        df = df[df['Condition'].isin(conditions)]
    if start_date:
        df = df[df["Date"] >= pd.to_datetime(start_date)]
    if end_date:
        df = df[df["Date"] <= pd.to_datetime(end_date)]

    if df.empty:
        print("No matching data found.")
    else:
        display(df)

    return df

def summarize_and_export():
    if not weather_data:
        print("No data to summarize or export.")
        return

    df = pd.DataFrame(weather_data)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')

    avg_temp = df["Temperature"].mean()
    max_temp = df["Temperature"].max()
    min_temp = df["Temperature"].min()

    print("📈 Weather Summary:")
    print(f"• Average Temperature: {avg_temp:.2f}°C")
    print(f"• Max Temperature: {max_temp:.2f}°C")
    print(f"• Min Temperature: {min_temp:.2f}°C")

    condition_counts = df["Condition"].value_counts()
    print("\n☁️ Weather Condition Counts:")
    for cond, count in condition_counts.items():
        print(f"• {cond}: {count} day(s)")

    print("\n📆 Average Temperature by Month:")
    df['Month'] = df['Date'].dt.to_period('M')
    monthly_avg = df.groupby('Month')['Temperature'].mean()
    for month, temp in monthly_avg.items():
        print(f"• {month}: {temp:.2f}°C")

    high_temp_days = df[df["Temperature"] > 35]
    print(f"\n🔥 Days Above 35°C: {len(high_temp_days)}")

    start_date = df["Date"].min().strftime('%Y-%m-%d')
    end_date = df["Date"].max().strftime('%Y-%m-%d')
    print(f"\n📆 Date Range: {start_date} to {end_date}")

    df.to_csv("weather_data.csv", index=False)
    print("✅ Data exported to 'weather_data.csv'")

In [40]:


def plot_temperature_trend():
    if not weather_data:
        print("No data to plot.")
        return
    df = pd.DataFrame(weather_data)
    df["Date"] = pd.to_datetime(df["Date"])
    df.sort_values("Date", inplace=True)

    plt.figure(figsize=(10,5))
    plt.plot(df["Date"], df["Temperature"], marker='o', linestyle='-', color='orange')
    plt.title("🌡️ Temperature Trend Over Time")
    plt.xlabel("Date")
    plt.ylabel("Temperature (°C)")
    plt.grid(True)
    plt.show()

    if not weather_data:
        print("No data to summarize.")
        return

    df = pd.DataFrame(weather_data)
    avg_temp = df["Temperature"].mean()
    print(f"📊 Average Temperature: {avg_temp:.2f}°C")

    df.to_csv("weather_data.csv", index=False)
    print("✅ Data exported to 'weather_data.csv'")

In [41]:
# Input Fields
from datetime import date
date_input = widgets.DatePicker(description="Date:", value=date.today())
temp_input = widgets.FloatText(description="Temperature (°C):")
condition_input = widgets.Dropdown(
    options=["Sunny", "Cloudy", "Rainy", "Windy", "Stormy", "Foggy", "Snowy"],
    description="Condition:"
)

add_button = widgets.Button(description="Add Data", button_style='success')
view_button = widgets.Button(description="View Data", button_style='info')
export_button = widgets.Button(description="Export & Summarize")
graph_button = widgets.Button(description="📈 Show Graph", button_style="info")
csv_input = widgets.Text(placeholder="Enter CSV file path", description="Load CSV:", value="weather_data.csv")
csv_button = widgets.Button(description="📂 Load Data", button_style="primary")
reset_button = widgets.Button(description="🔄 Reset Filters", button_style="danger")

output = widgets.Output()

In [42]:
condition_options = ["Sunny", "Cloudy", "Rainy", "Windy", "Stormy", "Foggy", "Snowy"]
condition_checkboxes = [widgets.Checkbox(value=False, description=cond) for cond in condition_options]

def get_selected_conditions():
    return [cb.description for cb in condition_checkboxes if cb.value]

checkbox_ui = widgets.HBox(condition_checkboxes)
start_date_picker = widgets.DatePicker(description="From:")
end_date_picker = widgets.DatePicker(description="To:")

In [43]:
def on_add_clicked(b):
    with output:
        clear_output()
    if date_input.value:
        date_str = date_input.value.strftime("%Y-%m-%d")
        add_weather_data(date_str, temp_input.value, condition_input.value)
    else:
        print("❌ Please select a date.")

def on_view_clicked(b):
    with output:
        clear_output()
        df = pd.DataFrame(weather_data)
        if df.empty:
            print("No data available.")
            return
        df["Date"] = pd.to_datetime(df["Date"])
        conditions = get_selected_conditions()
        if conditions:
            df = df[df["Condition"].isin(conditions)]
        if start_date_picker.value:
            df = df[df["Date"] >= pd.to_datetime(start_date_picker.value)]
        if end_date_picker.value:
            df = df[df["Date"] <= pd.to_datetime(end_date_picker.value)]
        if df.empty:
            print("No matching data found.")
        else:
            display(df)

def on_export_clicked(b):
    with output:
        clear_output()
        summarize_and_export()

def on_graph_clicked(b):
    with output:
        clear_output()
        df = filter_by_conditions_and_date(
            get_selected_conditions(),
            start_date_picker.value,
            end_date_picker.value
        )
        if df is None or df.empty:
            return
        df.sort_values("Date", inplace=True)
        plt.figure(figsize=(10,5))
        plt.plot(df["Date"], df["Temperature"], marker='o', linestyle='-', color='orange')
        plt.title("🌡️ Temperature Trend Over Time")
        plt.xlabel("Date")
        plt.ylabel("Temperature (°C)")
        plt.grid(True)
        plt.show()

def on_load_clicked(b):
    with output:
        clear_output()
        load_from_csv(csv_input.value)

def on_reset_clicked(b):
    for cb in condition_checkboxes:
        cb.value = False
    start_date_picker.value = None
    end_date_picker.value = None

add_button.on_click(on_add_clicked)
view_button.on_click(on_view_clicked)
export_button.on_click(on_export_clicked)
graph_button.on_click(on_graph_clicked)
csv_button.on_click(on_load_clicked)
reset_button.on_click(on_reset_clicked)

In [44]:
section_title = lambda text: widgets.HTML(f"<b style='font-size:16px'>{text}</b>")
spacer = lambda h=10: widgets.Box(layout=widgets.Layout(height=f'{h}px'))

load_tab = widgets.VBox([
    section_title("Load Existing Weather Data"),
    spacer(),
    csv_input,
    csv_button
], layout=widgets.Layout(padding='10px', border='1px solid lightgray'))

add_tab = widgets.VBox([
    section_title("Add New Weather Entry"),
    spacer(),
    date_input,
    temp_input,
    condition_input,
    spacer(5),
    add_button
], layout=widgets.Layout(padding='10px', border='1px solid lightgray'))

filters_ui = widgets.VBox([
    section_title("Filters"),
    spacer(5),
    checkbox_ui,
    widgets.HBox([start_date_picker, end_date_picker])
], layout=widgets.Layout(margin='0 0 10px 0'))

view_tab = widgets.VBox([
    filters_ui,
    widgets.HBox([view_button, reset_button])
], layout=widgets.Layout(padding='10px', border='1px solid lightgray'))

summary_tab = widgets.VBox([
    filters_ui,
    export_button,
    graph_button
], layout=widgets.Layout(padding='10px', border='1px solid lightgray'))

tab = widgets.Tab()
tab.children = [load_tab, add_tab, view_tab, summary_tab]
tab.set_title(0, '📂 Load CSV')
tab.set_title(1, '➕ Add Weather')
tab.set_title(2, '👁 View Data')
tab.set_title(3, '📊 Summary & Export')

display(tab, output)

Output()

⚠️ Date already exists! Please enter a new date.
⚠️ Date already exists! Please enter a new date.
